# Import libraries

In [1]:
import pandas as pd                 # pandas is a dataframe library
import matplotlib.pyplot as plt      # matplotlib.pyplot plots data

%matplotlib inline

# Read in data

In [2]:
df = pd.read_csv("./DeathRecords/new_data.csv")
del df['Id']

# Split into train and test

In [3]:
from sklearn.model_selection import train_test_split

feature_col_names = df.columns.values

x = df[feature_col_names].values     # predictor feature columns (8 X m)
split_test_size = 0.30

X_train, X_test = train_test_split(x, test_size=split_test_size, random_state=42)

# Categorical Bernoulli Naive Bayes

In [4]:
import sklearn.naive_bayes
import numpy as np
from scipy.misc import logsumexp

class CategoricalNB(sklearn.naive_bayes.BaseDiscreteNB):
    """Naive Bayes classifier for multivariate Categorical models.
    Like MultinomialNB, this classifier is suitable for discrete data. The
    difference is that while MultinomialNB works with occurrence counts,
    CategoricalNB is designed for categorical features.
    Read more in the :ref:`User Guide <categorical_naive_bayes>`.
    Parameters
    ----------
    alpha : float, optional (default=1.0)
        Additive (Laplace/Lidstone) smoothing parameter
        (0 for no smoothing).
    fit_prior : boolean, optional (default=True)
        Whether to learn class prior probabilities or not.
        If false, a uniform prior will be used.
    class_prior : array-like, size=[n_classes,], optional (default=None)
        Prior probabilities of the classes. If specified the priors are not
        adjusted according to the data.
    Attributes
    ----------
    class_log_prior_ : array, shape = [n_classes]
        Log probability of each class (smoothed).
    feature_log_prob_ : array of dictionaries, shape = [n_classes, n_features, n_feature_vals]
        Empirical log probability of features given a class, P(x_i|y).
    class_count_ : array, shape = [n_classes]
        Number of samples encountered for each class during fitting. This
        value is weighted by the sample weight when provided.
    feature_count_ : array, shape = [n_classes, n_features, num_feature_vals]
        Number of samples encountered for each (class, feature, feature val)
        during fitting. This value is weighted by the sample weight when
        provided.
    Examples
    --------
    >>> import numpy as np
    >>> X = np.random.randint(2, size=(6, 100))
    >>> Y = np.array([1, 2, 3, 4, 4, 5])
    >>> from sklearn.naive_bayes import CategoricalNB
    >>> clf = CategoricalNB()
    >>> clf.fit(X, Y)
    CategoricalNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
    >>> print(clf.predict(X[2:3]))
    [3]
    References
    ----------
    C.D. Manning, P. Raghavan and H. Schuetze (2008). Introduction to
    Information Retrieval. Cambridge University Press, pp. 234-265.
    http://nlp.stanford.edu/IR-book/html/htmledition/the-bernoulli-model-1.html
    A. McCallum and K. Nigam (1998). A comparison of event models for naive
    Bayes text classification. Proc. AAAI/ICML-98 Workshop on Learning for
    Text Categorization, pp. 41-48.
    V. Metsis, I. Androutsopoulos and G. Paliouras (2006). Spam filtering with
    naive Bayes -- Which naive Bayes? 3rd Conf. on Email and Anti-Spam (CEAS).
    """

    def __init__(self, alpha=1.0, binarize=.0, fit_prior=True,
                 class_prior=None,feature_space=None,output_space=None,n_classes=0,max_EM_iter=50):
        self.alpha = alpha
        self.binarize = binarize
        self.fit_prior = fit_prior
        self.class_prior = class_prior

        if class_prior is None:
            self.class_log_prior_ = None
        else:
            self.class_log_prior_ = np.log(class_prior)

        self.output_space = output_space
        self.map_output_space = {}
        self.n_classes = n_classes

        self.class_count_ = None
        self.feature_count_ = None

        if n_classes > 0:
            self._reset_class_count()
            
        if output_space is not None:
            self.init_output_space()
        self.feature_space = feature_space

        if feature_space is None:
            self.n_feature = None
            self.n_feature_vals = None
            self.map_feature_space = None
        else:
            self.n_features = len(feature_space)
            self.set_n_feature_vals()
            self.set_map_feature_space()
            self._reset_feature_count()

        self.delta_ = None
        self.max_EM_iter = max_EM_iter
        self.feature_log_prob_ = None


    def set_map_feature_space(self):
        self.map_feature_space = []
        for i in range(self.n_features):
            self.map_feature_space.append({})
            for j,v in zip(range(self.n_feature_vals[i]),self.feature_space[i]):
                self.map_feature_space[i][str(v)] = j
            
    def set_n_feature_vals(self):
        self.n_feature_vals = []
        for i in range(self.n_features):
            self.n_feature_vals.append(len(self.feature_space[i]))
                
    def _count(self, X, Y):
        """Count and smooth feature occurrences."""
        self.class_count_ += self._expected_class_count(Y)
        tmp_feature_count_ = self._expected_class_feature_count(X,Y)
        for k in range(self.n_classes):
            for i in range(self.n_features):
                self.feature_count_[k,i] += tmp_feature_count_[k,i]
        #self.feature_count_ += self._expected_class_feature_count(X,Y)

    def _M_step(self):
        self._update_feature_log_prob()
        self._update_class_log_prior(class_prior=self.class_prior)

    def _E_step(self,X,Y):
        m = self.n_classes
        num_samples = X.shape[0]
        self.delta_ = self.predict_proba(X)
        #print self.delta_.shape
        for l in range(num_samples):
            if Y[l] is not None:
                self.delta_[l,:] = np.zeros(self.n_classes)
                self.delta_[l,self.map_output_space[str(Y[l])]] = 1.0
        self._count(X, Y)
    
    def _expected_class_feature_count(self, X, Y):
        m = self.n_classes
        num_samples = X.shape[0]
        n = self.n_features
        count = np.empty((m, n),dtype=object)
        for c in range(m):
            for i in range(n):
                count[c,i] = np.zeros(self.n_feature_vals[i])
                
        for l in range(num_samples):
            if Y[l] is not None:
                for i in range(n):
                    count[self.map_output_space[str(Y[l])],i][self.map_feature_space[i][str(X[l,i])]] += 1.0
            else:
                for c in range(m):
                    for i in range(n):
                        count[c,i][self.map_feature_space[i][str(X[l,i])]] += self.delta_[l,c]
        return count

    def _expected_class_count(self, Y):
        m = self.n_classes
        count = np.zeros(m)
        n_samples = self.delta_.shape[0]
        for l in range(n_samples):
            if Y[l] is not None:
                count[self.map_output_space[str(Y[l])]] += 1.0
            else:
                count += self.delta_[l,:]                
        return count

    def _update_feature_log_prob(self):
        """Apply smoothing to raw counts and recompute log probabilities"""
        smoothed_fc = self.feature_count_
        for k in range(self.n_classes):
            for i in range(self.n_features):
                smoothed_fc[k,i] += self.alpha
                self.feature_log_prob_[k,i] = (np.log(smoothed_fc[k,i]) - np.log(np.sum(smoothed_fc[k,i])))

    def _joint_log_likelihood(self, X):
        """Calculate the posterior log probability of the samples X"""
        # check_is_fitted(self, "classes_")

        print("_joint_log_likelihood...")
        
        n_classes, n_features = self.feature_log_prob_.shape
        n_samples, n_features_X = X.shape

        if n_features_X != n_features:
            raise ValueError("Expected input with %d features, got %d instead"
                             % (n_features, n_features_X))

        jll = np.zeros((n_samples,n_classes))
        for l in range(n_samples):
            for c in range(n_classes):
                for i in range(n_features):
                    jll[l,c] += self.feature_log_prob_[c,i][self.map_feature_space[i][str(X[l,i])]]
            jll[l,:] += self.class_log_prior_

        print("Avg. log. likelihood: %f" % (np.mean(logsumexp(jll,axis=1))))
        print("done")
        return jll

    def set_feature_space(self, X):
        self.n_features = X.shape[1]
        self.feature_space = []
        for i in range(self.n_features):
            self.feature_space.append(list({str(v) for v in frozenset(X[:,i])}))
        self.set_n_feature_vals()
        self.set_map_feature_space()
        self._reset_feature_count()

    def init_output_space(self):
        self.n_classes = len(self.output_space)
        self.map_output_space = {}
        for k,s in zip(range(self.n_classes),[str(v) for v in self.output_space]):
            self.map_output_space[s] = k
            
    def set_output_space(self, y):
        S = {str(c) for c in y.unique()}
        self.output_space = list(S)
        self.init_output_space()

    def _rand_unif_dist(self, n_vals):
        vals = np.zeros(n_vals+1)
        vals[n_vals] = 1.0
        vals[1:n_vals] = np.sort(np.random.rand(n_vals-1))
        return np.diff(vals)

    def rand_init_class_log_prior(self):
        self.class_log_prior_ = np.log(self._rand_unif_dist(self.n_classes))

    def rand_init_feature_prob(self):
        self.feature_log_prob_ = np.empty((self.n_classes, self.n_features),dtype=object)
        for c in range(self.n_classes):
            for i in range(self.n_features):
                self.feature_log_prob_[c,i] = np.log(self._rand_unif_dist(self.n_feature_vals[i]))

    def _reset_count(self):
        self._reset_class_count()
        self._reset_feature_count()

    def _reset_feature_count(self):
        self.feature_count_ = np.empty((self.n_classes, self.n_features),dtype=object)
        for k in range(self.n_classes):
            for i in range(self.n_features):
                self.feature_count_[k,i] = np.zeros(self.n_feature_vals[i])

    def _reset_class_count(self):
        self.class_count_ = np.zeros(self.n_classes)
                
    def fit(self, X, y):
        """Fit Naive Bayes classifier according to X, y
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        y : array-like, shape = [n_samples]
            Target values.
        Returns
        -------
        self : object
            Returns self.
        """
        
        if self.feature_space is None:
            self.set_feature_space(X)

        if self.output_space is None:
            self.set_output_space(y)

        self.classes_ = np.array(self.output_space)
        
        if self.class_log_prior_ is None:
            self.rand_init_class_log_prior()

        if self.feature_log_prob_ is None:
            self.rand_init_feature_prob()
        
        # Run Expectation-Maximization (EM) Algorithm

        print(np.exp(self.class_log_prior_))
        
        for t in range(self.max_EM_iter):
            #print t
            self._E_step(X,y)
            self._M_step()
            print(np.exp(self.class_log_prior_))
            self._reset_count()
            
        return self

In [ ]:
# Import libraries
import pandas as pd                 # pandas is a dataframe library
import matplotlib.pyplot as plt      # matplotlib.pyplot plots data
import numpy as np
from scipy.misc import logsumexp

# %matplotlib inline

# Read in data

df = pd.read_csv("./DeathRecords/new_data.csv")
del df['Id']

# Split into train and test

from sklearn.model_selection import train_test_split

feature_col_names = df.columns.values

x = df[feature_col_names].values     # predictor feature columns (8 X m)
split_test_size = 0.30

X_train, X_test = train_test_split(x, test_size=split_test_size, random_state=42)

X_train_cv, X_test_cv = train_test_split(X_train, test_size=split_test_size)

# Categorical Bernoulli Naive Bayes

# from categorical_nb import CategoricalNB

Y_train_cv = []
#for row in range(len(df)):
for l in range(X_train_cv.shape[0]):
    Y_train_cv.append(None)

#print X_train.shape
# s = X_train[:,1]

# print s, len(s)

# S = frozenset(s)

# print S, len(S)

# S_asstr = {str(v) for v in S}

# print S_asstr, len(S_asstr)

# S_aslist = list(S_asstr)

# print S_aslist, len(S_aslist)

#print [str(v) for v in range(3)]
    
hold_out_avg_log_likelihood = []    

# try doing the binary search yourself
# k = number of clusters
for k in np.arange(10,41,10):
    # n_classes = num clusters, output_space = assigning the cluster a label
    nb_model = CategoricalNB(n_classes=k,output_space=[str(v) for v in range(k)],max_EM_iter=1)
    
    nb_model.set_feature_space(x)

    # iter = num of iterations

    prev_cv_avg_log_likelihood = float('-inf')
    for iter in np.arange(1,10000000):
        nb_model.fit(X_train_cv[0:10000,:], y=Y_train_cv[0:10000])
        
        cv_jll = nb_model._joint_log_likelihood(X_test_cv[0:10000,:])
        cv_avg_log_likelihood = np.mean(logsumexp(cv_jll,axis=1))
        result = [k, iter, cv_avg_log_likelihood]
        hold_out_avg_log_likelihood.append(result)
        print(result)
        if cv_avg_log_likelihood > prev_cv_avg_log_likelihood:
            prev_cv_avg_log_likelihood = cv_avg_log_likelihood
        else:
            break
        
# print(hold_out_avg_log_likelyhood)
# find best k and iters from hold_out_log_likelihood (3rd element in lisr)

#hold_out_log_likelihood = np.matrix(hold_out_avg_log_likelyhood, dtype=np.float64)
#k_best, iter_best, cv_avg_log_likelihood_best = hold_out_log_likelihood_matrix[np.argmax(hold_out_log_likelihood_matrix[:,2]),:]
k_best, iter_best, cv_avg_log_likelihood_best = hold_out_avg_log_likelihood[np.argmax(hold_out_avg_log_likelihood[:][2])][:]

nb_model_best = CategoricalNB(n_classes=k_best,output_space=[str(v) for v in range(k_best)],max_EM_iter=iter_best)
nb_model_best.set_feature_space(x)

nb_model_best.fit(X_train, y=Y_train)

import pickle

pickle.dump('~/Documents/best_nb_model.pkl',nb_model)

In [70]:
feature_values = []
for feature in df.columns:
    feature_values.append(df[feature].unique())

Y = []
for row in range(len(df)):
    Y.append(None)
print(len(Y))

2631171
